In [ ]:
%pip install langgraph

In [22]:
# Install nest_asyncio to allow asyncio.run() in environments with a running event loop (like Colab/Jupyter)
!pip install nest_asyncio --quiet

from langgraph.graph import StateGraph, START, END
# Removed unused 'entrypoint' from import
from langgraph.func import task
from typing import TypedDict
import asyncio
import nest_asyncio

# Apply the patch to allow nested event loops
nest_asyncio.apply()

class ParallelState(TypedDict):
    name: str

async def parse_resume(state: ParallelState) -> dict:
    print(f"📄 Parsing resume for {state['name']} (2s)...")
    await asyncio.sleep(2)
    print(f"   ✅ Resume parsed")
    return {"resume_parsed": True}

async def check_background(state: ParallelState) -> dict:
    print(f"🔍 Starting background check (3s)...")
    await asyncio.sleep(3)
    print(f"   ✅ Background check complete")
    return {"background_check": "Clear"}

async def check_references(state: ParallelState) -> dict:
    print(f"📞 Starting reference check (2s)...")
    await asyncio.sleep(2)
    print(f"   ✅ Reference check complete")
    return {"references": "Positive"}

async def make_decision(state: ParallelState) -> dict:
    print(f"📋 Making final decision for {state['name']}...")
    return {"decision": "HIRED"}

def create_parallel_graph():
    workflow = StateGraph(ParallelState)
    workflow.add_node("parse", parse_resume)
    workflow.add_node("background", check_background)
    workflow.add_node("references", check_references)
    workflow.add_node("decide", make_decision)
    workflow.add_edge(START, "parse")
    workflow.add_edge("parse", "background")
    workflow.add_edge("parse", "references")
    workflow.add_edge("background", "decide")
    workflow.add_edge("references", "decide")
    workflow.add_edge("decide", END)
    return workflow.compile()

# Removed @task decorator
async def parse_resume_task(name: str) -> dict:
    print(f"📄 Parsing resume for {name} (2s)...")
    await asyncio.sleep(2)
    print(f"   ✅ Resume parsed")
    return {"parsed": True}

# Removed @task decorator
async def check_background_task(name: str) -> dict:
    print(f"🔍 Starting background check (3s)...")
    await asyncio.sleep(3)
    print(f"   ✅ Background complete")
    return {"status": "Clear"}

# Removed @task decorator
async def check_references_task(name: str) -> dict:
    print(f"📞 Starting reference check (2s)...")
    await asyncio.sleep(2)
    print(f"   ✅ References complete")
    return {"status": "Positive"}

# Removed @entrypoint decorator
async def screen_candidate_parallel(name: str) -> str:
    print(f"\n🎯 Starting screening for {name}...")
    await parse_resume_task(name)

    print("--- Running background and reference checks IN PARALLEL ---")
    background_result, reference_result = await asyncio.gather(
        check_background_task(name),
        check_references_task(name)
    )

    print(f"📋 Making final decision for {name}...")
    return f"HIRED - Background: {background_result['status']}, Refs: {reference_result['status']}"

async def main():
    print("="*70)
    print("GRAPH API - ASYNC WITH PARALLEL EXECUTION")
    print("="*70)
    graph_app = create_parallel_graph()
    start_graph = asyncio.get_event_loop().time()
    await graph_app.ainvoke({"name": "David Lee"})
    total_graph = asyncio.get_event_loop().time() - start_graph
    print(f"\n⏱️  Graph API Total Time: {total_graph:.1f} seconds")
    print(f"   (Expected: 2s + max(3s, 2s) = 5s)")

    print("\n" + "="*70)
    print("FUNCTIONAL API - ASYNC WITH PARALLEL EXECUTION")
    print("="*70)
    start_func = asyncio.get_event_loop().time()
    # CORRECTED LINE: Use .invoke() for the Functional API's async entrypoint
    await screen_candidate_parallel(name="Emma Wilson") # Directly await the async function
    total_func = asyncio.get_event_loop().time() - start_func
    print(f"\n⏱️  Functional API Total Time: {total_func:.1f} seconds")
    print(f"   (Expected: 2s + max(3s, 2s) = 5s)")

if __name__ == "__main__":
    asyncio.run(main())

GRAPH API - ASYNC WITH PARALLEL EXECUTION
📄 Parsing resume for David Lee (2s)...
   ✅ Resume parsed
🔍 Starting background check (3s)...
📞 Starting reference check (2s)...
   ✅ Reference check complete
   ✅ Background check complete
📋 Making final decision for David Lee...

⏱️  Graph API Total Time: 5.0 seconds
   (Expected: 2s + max(3s, 2s) = 5s)

FUNCTIONAL API - ASYNC WITH PARALLEL EXECUTION

🎯 Starting screening for Emma Wilson...
📄 Parsing resume for Emma Wilson (2s)...
   ✅ Resume parsed
--- Running background and reference checks IN PARALLEL ---
🔍 Starting background check (3s)...
📞 Starting reference check (2s)...
   ✅ References complete
   ✅ Background complete
📋 Making final decision for Emma Wilson...

⏱️  Functional API Total Time: 5.0 seconds
   (Expected: 2s + max(3s, 2s) = 5s)
